In [5]:
import numpy as np
import random, math
import copy
import sys
import math

A = np.array([[ 3,  8,  1,  2], [ 3, -2,  5,  4], [ 4, -3,  7,  0],  [ 4,  2, -2,  0]])
b= np.array([-265, 1177, 1162, -164])
# x = [52 -87  99  88] # Solution vector
# xi = qi1 + 2qi2 + 4qi3 - qi4 - 2qi5 - 4qi6 
# Subrange interval = 15

T = np.array([45, -90,  105,  90])
# q = ([1, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0]. [0. 0, 0, 0, 1, 1]. [0, 0, 0, 0, 1, 0])
# Please read the minimum energy carefully.
# We will updae q1 to q01 at the next version

c = np.int64(b - np.dot(A,T))
(Dimension, Dimension_same) = A.shape
qubits = 3

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[ -206.   200.   400.     0.     0.     0.    28.    56.   112.   -28.
    -56.  -112.    76.   152.   304.   -76.  -152.  -304.    36.    72.
    144.   -36.   -72.  -144.]
 [    0.  -312.   800.     0.     0.     0.    56.   112.   224.   -56.
   -112.  -224.   152.   304.   608.  -152.  -304.  -608.    72.   144.
    288.   -72.  -144.  -288.]
 [    0.     0.  -224.     0.     0.     0.   112.   224.   448.  -112.
   -224.  -448.   304.   608.  1216.  -304.  -608. -1216.   144.   288.
    576.  -144.  -288.  -576.]
 [    0.     0.     0.   306.   200.   400.   -28.   -56.  -112.    28.
     56.   112.   -76.  -152.  -304.    76.   152.   304.   -36.   -72.
   -144.    36.    72.   144.]
 [    0.     0.     0.     0.   712.   800.   -56.  -112.  -224.    56.
    112.   224.  -152.  -304.  -608.   152.   304.   608.   -72.  -144.
   -288.    72.   144.   288.]
 [    0.     0.     0.     0.     0.  1824.  -112.  -224.  -448.   112.
    224.   448.  -304.  -608. -1216.   

In [4]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-206.0, ('q2','q2'):-312.0, ('q3','q3'):-224.0, ('q4','q4'):306.0, ('q5','q5'):712.0, ('q6','q6'):1824.0, ('q7','q7'):-893.0, ('q8','q8'):-1624.0, ('q9','q9'):-2600.0, ('q10','q10'):1055.0, ('q11','q11'):2272.0, ('q12','q12'):5192.0, ('q13','q13'):745.0, ('q14','q14'):1648.0, ('q15','q15'):3928.0, ('q16','q16'):-587.0, ('q17','q17'):-1016.0, ('q18','q18'):-1400.0, ('q19','q19'):64.0, ('q20','q20'):168.0, ('q21','q21'):496.0, ('q22','q22'):-24.0, ('q23','q23'):-8.0, ('q24','q24'):144.0}

quadratic = {('q1','q2'):200.0, ('q1','q3'):400.0, ('q1','q7'):28.0, ('q1','q8'):56.0, ('q1','q9'):112.0, ('q1','q10'):-28.0, ('q1','q11'):-56.0, ('q1','q12'):-112.0, ('q1','q13'):76.0, ('q1','q14'):152.0, ('q1','q15'):304.0, ('q1','q16'):-76.0, ('q1','q17'):-152.0, ('q1','q18'):-304.0, ('q1','q19'):36.0, ('q1','q20'):72.0, ('q1','q21'):144.0, ('q1','q22'):-36.0, ('q1','q23'):-72.0, ('q1','q24'):-144.0, ('q2','q3'):800.0, ('q2','q7'):56.0, ('q2','q8'):112.0, ('q2','q9'):224.0, ('q2','q10'):-56.0, ('q2','q11'):-112.0, ('q2','q12'):-224.0, ('q2','q13'):152.0, ('q2','q14'):304.0, ('q2','q15'):608.0, ('q2','q16'):-152.0, ('q2','q17'):-304.0, ('q2','q18'):-608.0, ('q2','q19'):72.0, ('q2','q20'):144.0, ('q2','q21'):288.0, ('q2','q22'):-72.0, ('q2','q23'):-144.0, ('q2','q24'):-288.0, ('q3','q7'):112.0, ('q3','q8'):224.0, ('q3','q9'):448.0, ('q3','q10'):-112.0, ('q3','q11'):-224.0, ('q3','q12'):-448.0, ('q3','q13'):304.0, ('q3','q14'):608.0, ('q3','q15'):1216.0, ('q3','q16'):-304.0, ('q3','q17'):-608.0, ('q3','q18'):-1216.0, ('q3','q19'):144.0, ('q3','q20'):288.0, ('q3','q21'):576.0, ('q3','q22'):-144.0, ('q3','q23'):-288.0, ('q3','q24'):-576.0, ('q4','q5'):200.0, ('q4','q6'):400.0, ('q4','q7'):-28.0, ('q4','q8'):-56.0, ('q4','q9'):-112.0, ('q4','q10'):28.0, ('q4','q11'):56.0, ('q4','q12'):112.0, ('q4','q13'):-76.0, ('q4','q14'):-152.0, ('q4','q15'):-304.0, ('q4','q16'):76.0, ('q4','q17'):152.0, ('q4','q18'):304.0, ('q4','q19'):-36.0, ('q4','q20'):-72.0, ('q4','q21'):-144.0, ('q4','q22'):36.0, ('q4','q23'):72.0, ('q4','q24'):144.0, ('q5','q6'):800.0, ('q5','q7'):-56.0, ('q5','q8'):-112.0, ('q5','q9'):-224.0, ('q5','q10'):56.0, ('q5','q11'):112.0, ('q5','q12'):224.0, ('q5','q13'):-152.0, ('q5','q14'):-304.0, ('q5','q15'):-608.0, ('q5','q16'):152.0, ('q5','q17'):304.0, ('q5','q18'):608.0, ('q5','q19'):-72.0, ('q5','q20'):-144.0, ('q5','q21'):-288.0, ('q5','q22'):72.0, ('q5','q23'):144.0, ('q5','q24'):288.0, ('q6','q7'):-112.0, ('q6','q8'):-224.0, ('q6','q9'):-448.0, ('q6','q10'):112.0, ('q6','q11'):224.0, ('q6','q12'):448.0, ('q6','q13'):-304.0, ('q6','q14'):-608.0, ('q6','q15'):-1216.0, ('q6','q16'):304.0, ('q6','q17'):608.0, ('q6','q18'):1216.0, ('q6','q19'):-144.0, ('q6','q20'):-288.0, ('q6','q21'):-576.0, ('q6','q22'):144.0, ('q6','q23'):288.0, ('q6','q24'):576.0, ('q7','q8'):324.0, ('q7','q9'):648.0, ('q7','q13'):-54.0, ('q7','q14'):-108.0, ('q7','q15'):-216.0, ('q7','q16'):54.0, ('q7','q17'):108.0, ('q7','q18'):216.0, ('q7','q19'):16.0, ('q7','q20'):32.0, ('q7','q21'):64.0, ('q7','q22'):-16.0, ('q7','q23'):-32.0, ('q7','q24'):-64.0, ('q8','q9'):1296.0, ('q8','q13'):-108.0, ('q8','q14'):-216.0, ('q8','q15'):-432.0, ('q8','q16'):108.0, ('q8','q17'):216.0, ('q8','q18'):432.0, ('q8','q19'):32.0, ('q8','q20'):64.0, ('q8','q21'):128.0, ('q8','q22'):-32.0, ('q8','q23'):-64.0, ('q8','q24'):-128.0, ('q9','q13'):-216.0, ('q9','q14'):-432.0, ('q9','q15'):-864.0, ('q9','q16'):216.0, ('q9','q17'):432.0, ('q9','q18'):864.0, ('q9','q19'):64.0, ('q9','q20'):128.0, ('q9','q21'):256.0, ('q9','q22'):-64.0, ('q9','q23'):-128.0, ('q9','q24'):-256.0, ('q10','q11'):324.0, ('q10','q12'):648.0, ('q10','q13'):54.0, ('q10','q14'):108.0, ('q10','q15'):216.0, ('q10','q16'):-54.0, ('q10','q17'):-108.0, ('q10','q18'):-216.0, ('q10','q19'):-16.0, ('q10','q20'):-32.0, ('q10','q21'):-64.0, ('q10','q22'):16.0, ('q10','q23'):32.0, ('q10','q24'):64.0, ('q11','q12'):1296.0, ('q11','q13'):108.0, ('q11','q14'):216.0, ('q11','q15'):432.0, ('q11','q16'):-108.0, ('q11','q17'):-216.0, ('q11','q18'):-432.0, ('q11','q19'):-32.0, ('q11','q20'):-64.0, ('q11','q21'):-128.0, ('q11','q22'):32.0, ('q11','q23'):64.0, ('q11','q24'):128.0, ('q12','q13'):216.0, ('q12','q14'):432.0, ('q12','q15'):864.0, ('q12','q16'):-216.0, ('q12','q17'):-432.0, ('q12','q18'):-864.0, ('q12','q19'):-64.0, ('q12','q20'):-128.0, ('q12','q21'):-256.0, ('q12','q22'):64.0, ('q12','q23'):128.0, ('q12','q24'):256.0, ('q13','q14'):316.0, ('q13','q15'):632.0, ('q13','q19'):44.0, ('q13','q20'):88.0, ('q13','q21'):176.0, ('q13','q22'):-44.0, ('q13','q23'):-88.0, ('q13','q24'):-176.0, ('q14','q15'):1264.0, ('q14','q19'):88.0, ('q14','q20'):176.0, ('q14','q21'):352.0, ('q14','q22'):-88.0, ('q14','q23'):-176.0, ('q14','q24'):-352.0, ('q15','q19'):176.0, ('q15','q20'):352.0, ('q15','q21'):704.0, ('q15','q22'):-176.0, ('q15','q23'):-352.0, ('q15','q24'):-704.0, ('q16','q17'):316.0, ('q16','q18'):632.0, ('q16','q19'):-44.0, ('q16','q20'):-88.0, ('q16','q21'):-176.0, ('q16','q22'):44.0, ('q16','q23'):88.0, ('q16','q24'):176.0, ('q17','q18'):1264.0, ('q17','q19'):-88.0, ('q17','q20'):-176.0, ('q17','q21'):-352.0, ('q17','q22'):88.0, ('q17','q23'):176.0, ('q17','q24'):352.0, ('q18','q19'):-176.0, ('q18','q20'):-352.0, ('q18','q21'):-704.0, ('q18','q22'):176.0, ('q18','q23'):352.0, ('q18','q24'):704.0, ('q19','q20'):80.0, ('q19','q21'):160.0, ('q20','q21'):320.0, ('q22','q23'):80.0, ('q22','q24'):160.0, ('q23','q24'):320.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
1    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0      17 ...
474  1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
3    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       3 ...
4    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4349.0       7 ...
5    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       4 ...
6    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       4 ...
7    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4344.0       3 ...
8    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       1 ...
10   0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4341.0       2 ...
12   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       4 ...
13   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...
14   1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       7 ...

In [6]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
51   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
65   0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       1 ...
2    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4351.0       5 ...
72   0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4351.0       1 ...
74   0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4349.0       1 ...
3    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       7 ...
4    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       4 ...
5    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4334.0       2 ...
85   1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
102  1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
103  1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       2 ...
169  1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...

In [7]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
100  1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
115  0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       2 ...
126  0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4349.0       2 ...
128  0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       2 ...
130  0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4341.0       3 ...
132  0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4334.0       1 ...
133  1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...
134  1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
139  1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4328.0       2 ...
58   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       3 ...
142  1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4318.0       1 ...
143  1   0   0   0   0   0   0   0   1   1   0  1 ...  1 -4318.0       1 ...

In [8]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
0    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       1 ...
1    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       1 ...
2    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       2 ...
3    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4344.0       1 ...
4    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       1 ...
5    1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       5 ...
6    1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       2 ...
474  1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
7    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4328.0       2 ...
8    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4320.0       1 ...
9    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       1 ...
10   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       1 ...

In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
0    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
1    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       3 ...
2    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       1 ...
3    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       3 ...
673  0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       1 ...
634  0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4351.0       1 ...
4    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4349.0       5 ...
5    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       3 ...
6    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       2 ...
7    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       2 ...
8    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4334.0       1 ...
9    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       3 ...

In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
3    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
7    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4351.0       1 ...
8    1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
9    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...
10   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       2 ...
11   1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4320.0       2 ...
12   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       1 ...
13   1   0   0   0   0   0   0   0   1   1   0  1 ...  1 -4314.0       1 ...
14   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4314.0       1 ...
648  1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4314.0       1 ...
17   1   0   0   0   0   0   0   1   1   1   1  1 ...  0 -4293.0       2 ...
444  1   0   0   0   0   0   0   1   1   1   1  1 ...  0 -4293.0       1 ...

In [11]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
22   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
23   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       2 ...
26   0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       3 ...
30   0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4351.0       1 ...
32   0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       2 ...
33   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       3 ...
34   1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4344.0       1 ...
427  1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       1 ...
36   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...
39   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4320.0       1 ...
40   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       1 ...
41   1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4318.0       1 ...

In [12]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
0    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       1 ...
1    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       3 ...
2    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       6 ...
569  1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       1 ...
3    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       1 ...
673  0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       1 ...
4    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4349.0       3 ...
5    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       3 ...
6    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       3 ...
527  0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       1 ...
7    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4344.0       3 ...
8    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       5 ...

In [13]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
0    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       1 ...
1    1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       1 ...
2    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4328.0       1 ...
3    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4328.0       1 ...
4    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4319.0       1 ...
5    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4315.0       3 ...
6    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4314.0       1 ...
7    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4303.0       2 ...
8    0   0   0   0   0   0   0   0   1   1   1  1 ...  0 -4301.0       1 ...
9    1   0   0   0   0   0   0   1   0   1   0  0 ...  1 -4301.0       1 ...
10   0   0   0   0   0   0   0   0   0   1   0  1 ...  1 -4296.0       1 ...
11   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4293.0       1 ...

In [14]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9  energy num_oc. ...
1    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4399.0       6 ...
2    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       3 ...
3    1   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4379.0       2 ...
5    0   0   0   0   0   0   0   0   1   1   0  1 ...  0 -4365.0       1 ...
6    0   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4347.0       3 ...
7    0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4346.0       3 ...
8    1   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4344.0       4 ...
9    1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4344.0       2 ...
10   0   0   0   0   0   0   0   1   0   1   0  1 ...  0 -4334.0       1 ...
11   1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0      15 ...
12   1   0   0   0   0   0   0   1   1   1   0  1 ...  0 -4333.0       4 ...
13   1   0   0   0   0   0   0   1   0   1   0  1 ...  1 -4333.0       3 ...